In [2]:
!pip install pandas numpy scikit-learn nltk pdfplumber

In [3]:
import os
import pdfplumber
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import re

# Download stopwords if not available
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dom\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [1]:
import os
print(os.getcwd())

C:\Users\Dom\your_project


In [3]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text
    pdf_path = "resumes"
   

In [4]:
# Load all resumes
resume_texts = {}
for filename in os.listdir("resumes"):
    if filename.endswith(".pdf"):
        resume_texts[filename] = extract_text_from_pdf(os.path.join("resumes", filename))

# Load job description
with open("job_description.txt", "r", encoding="utf-8") as f:
    job_description = f.read()

NameError: name 'pdfplumber' is not defined

In [5]:
!pip install pdfplumber


In [6]:
# Load all resumes
import pdfplumber
resume_texts = {}
for filename in os.listdir("resumes"):
    if filename.endswith(".pdf"):
        resume_texts[filename] = extract_text_from_pdf(os.path.join("resumes", filename))

# Load job description
with open("job_description.txt", "r", encoding="utf-8") as f:
    job_description = f.read()

In [19]:
import re
import nltk
from nltk.corpus import stopwords

# Download stopwords the first time you use NLTK
nltk.download('stopwords')
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join(text.split())
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
    
# Extract skills from text
def extract_skills(text, skills_list):
    found = []
    text_lower = text.lower()
    for skill in skills_list:
        if skill in text_lower:
            found.append(skill)
    return ", ".join(found) if found else "No major skills found"

cleaned_resumes = {name: clean_text(text) for name, text in resume_texts.items()}
cleaned_jd = clean_text(job_description)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
skills_list = [
    "sql", "python", "pandas", "numpy", "excel", "power bi", "tableau",
    "data analysis", "data visualization", "machine learning",
    "dashboard", "business intelligence", "statistics", "etl",
    "reporting", "data cleaning"
]

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
vectorizer = TfidfVectorizer()
all_texts = [cleaned_jd] + list(cleaned_resumes.values())
tfidf_matrix = vectorizer.fit_transform(all_texts)

similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

results = pd.DataFrame({
    'Candidate': list(cleaned_resumes.keys()),
    'Match Score (%)': (similarity_scores * 100).round(2)
})

results["Key Skills Found"] = results["Candidate"].apply(
    lambda x: extract_skills(cleaned_resumes[x], skills_list)
)

# Sort by Match Score
results = results.sort_values(by="Match Score (%)", ascending=False).reset_index(drop=True)
results

,Candidate,Match Score (%),Key Skills Found
0,resume_alice.pdf,39.14,"sql, python, power bi, data analysis, machine ..."
1,resume_rahul.pdf,8.91,"sql, excel, tableau, dashboard, reporting"
2,resume_sneha.pdf,0.73,No major skills found
